In [116]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

In [117]:
base_path = 'data'
activities = ['running', 'idle', 'walking', 'stairs']
data_list = []
for activity in activities:
    folder_path = os.path.join(base_path, activity)
    for filename in os.listdir(folder_path):
        if filename.endswith('.csv'):
            file_path = os.path.join(folder_path, filename)
            df = pd.read_csv(file_path, header=None, names=['x', 'y', 'z'])
            df['activity'] = activity
            data_list.append(df)
all_data = pd.concat(data_list, ignore_index=True)
all_data.to_csv("prepare_data", index = False)
df = pd.read_csv('prepare_data')
df_cleaned = all_data[~((df['x'] == 'accelerometer_X') & (df['y'] == 'accelerometer_Y') & (df['z'] == 'accelerometer_Z'))]
df_cleaned["x"] = pd.to_numeric(df_cleaned["x"], errors='coerce').astype('float64')
df_cleaned["y"] = pd.to_numeric(df_cleaned["y"], errors='coerce').astype('float64')
df_cleaned["z"] = pd.to_numeric(df_cleaned["z"], errors='coerce').astype('float64')

C:\Users\kybav\AppData\Local\Temp\ipykernel_28424\214068956.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned["x"] = pd.to_numeric(df_cleaned["x"], errors='coerce').astype('float64')
C:\Users\kybav\AppData\Local\Temp\ipykernel_28424\214068956.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned["y"] = pd.to_numeric(df_cleaned["y"], errors='coerce').astype('float64')
C:\Users\kybav\AppData\Local\Temp\ipykernel_28424\214068956.py:18: SettingWithCopyWarning: 
A value is trying to be se

In [118]:
def extract_time_features(window):
    features = {}
    features['mean_x'] = window['x'].mean()
    features['mean_y'] = window['y'].mean()
    features['mean_z'] = window['z'].mean()
    features['std_x'] = window['x'].std()
    features['std_y'] = window['y'].std()
    features['std_z'] = window['z'].std()
    features['max_x'] = window['x'].max()
    features['max_y'] = window['y'].max()
    features['max_z'] = window['z'].max()
    features['min_x'] = window['x'].min()
    features['min_y'] = window['y'].min()
    features['min_z'] = window['z'].min()
    return pd.Series(features)

window_size = 128  
features_list = []
labels = []

for i in range(0, len(df_cleaned) - window_size, window_size):
    window = df_cleaned.iloc[i:i + window_size]
    features = extract_time_features(window)
    features_list.append(features)
    labels.append(window['activity'].mode()[0])  # Найпоширеніша активність у вікні
features_df = pd.DataFrame(features_list)
labels = pd.Series(labels)

In [119]:
X_train, X_test, y_train, y_test = train_test_split(features_df, labels, test_size=0.2, random_state=42)

In [120]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [121]:
svm_model = SVC(kernel='linear')
svm_model.fit(X_train_scaled, y_train)
svm_pred = svm_model.predict(X_test_scaled)

In [122]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_test)

In [123]:
print("SVM Accuracy:", accuracy_score(y_test, svm_pred))
print("Random Forest Accuracy:", accuracy_score(y_test, rf_pred))
print("\nClassification Report (SVM):\n", classification_report(y_test, svm_pred))
print("\nClassification Report (Random Forest):\n", classification_report(y_test, rf_pred))

SVM Accuracy: 0.9834983498349835
Random Forest Accuracy: 0.9867986798679867

Classification Report (SVM):
               precision    recall  f1-score   support

        idle       1.00      1.00      1.00        51
     running       1.00      1.00      1.00       150
      stairs       0.75      0.67      0.71         9
     walking       0.97      0.98      0.97        93

    accuracy                           0.98       303
   macro avg       0.93      0.91      0.92       303
weighted avg       0.98      0.98      0.98       303


Classification Report (Random Forest):
               precision    recall  f1-score   support

        idle       1.00      1.00      1.00        51
     running       1.00      1.00      1.00       150
      stairs       0.86      0.67      0.75         9
     walking       0.97      0.99      0.98        93

    accuracy                           0.99       303
   macro avg       0.96      0.91      0.93       303
weighted avg       0.99      0.99    